<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/09_9_RAG_API.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Upstage RAG API (experimental)



## Environment

In [2]:
!pip install -qU langchain-upstage langchain python-dotenv openai rich


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Environment variables

Set up environment variables 
* UPSTAGE_API_KEY


In [3]:
# @title set API key
from pprint import pprint
from rich import print as rprint

import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [14]:
rag_base_url = "https://experimental-api.x.upstage.ai/rag-api/"

# Plase chose your model name
rag_model_name = "sung-pro-1001"

In [24]:
from openai import OpenAI


client = OpenAI(base_url=rag_base_url, api_key=os.environ["UPSTAGE_API_KEY"])

print("Building/Rebuilding the index...")
directory = "pdfs"
# Upload all files in the pdfs folder
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    with open(file_path, "rb") as f:
        file_response = client.files.create(
            file=f, purpose="assistants", extra_body={"model_name": rag_model_name}
        )
        rprint(file_response)

print("Index build/rebuild submission completed.")

public_model_name = file_response.public_model_name
print("Public Mode Name:", public_model_name)

Building/Rebuilding the index...


FileObject(
    id='file-70',
    bytes=418872,
    created_at=1727795097,
    filename='solar_paper.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='sung-pro-1001',
    public_model_name='36127e427ea30842d7f1',
    message="File 'solar_paper.pdf' has already been indexed for this model. The existing indexed version will be 
used."
)

FileObject(
    id='file-81',
    bytes=137425,
    created_at=1727795097,
    filename='solar_sample.pdf',
    object='file',
    purpose='assistants',
    status='queued',
    status_details=None,
    model_name='sung-pro-1001',
    public_model_name='36127e427ea30842d7f1'
)

FileObject(
    id='file-72',
    bytes=878624,
    created_at=1727795097,
    filename='docai.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='sung-pro-1001',
    public_model_name='36127e427ea30842d7f1',
    message="File 'docai.pdf' has already been indexed for this model. The existing indexed version will be used."
)

FileObject(
    id='file-73',
    bytes=4432693,
    created_at=1727795098,
    filename='kim-tse-2008.pdf',
    object='file',
    purpose='assistants',
    status='ready',
    status_details=None,
    model_name='sung-pro-1001',
    public_model_name='36127e427ea30842d7f1',
    message="File 'kim-tse-2008.pdf' has already been indexed for this model. The existing indexed version will be 
used."
)

Index build/rebuild submission completed.
Public Mode Name: 36127e427ea30842d7f1


In [16]:
# Import necessary libraries
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(model=rag_model_name, base_url=rag_base_url)

# Send a query to the chat model
question = "What's the Document AI?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

AIMessage(
    content='According to the key-value pairs provided, "Document AI" is Upstage Document AI, which is comprised of
three solutions: Layout Analyzer, Key Information Extractor, and Document OCR. The Layout Analyzer function is to 
extract layouts, tables, and figures from any document. The Key Information Extractor function is to extract key 
information from target documents. The Document OCR function is to extract all text from any document.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': 'sung-pro-1001',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-89168b5c-f869-417f-a6dc-bf8cfcfa6e4e-0'
)

# Upstage RAG-API Configuration Options

The `Chat` object is initialized with several options to customize its behavior:

1. `api_key`: The API key for authentication. Get from https://console.upstage.ai/

2. `model`: The name of the RAG (Retrieval-Augmented Generation) model to be used.

3. `base_url`: The base URL for the RAG API.

4. `extra_body`: A dictionary containing additional parameters for the API request:

   - `hybrid_search` (bool): When set to `True`, enables a combination of semantic and keyword-based search for improved retrieval accuracy.
   
   - `contextual_query` (bool): If `True`, the system considers the context of previous interactions when processing the current query.
   
   - `contextual_chunk` (bool): When enabled, the system retrieves and processes information in context-aware chunks, potentially improving relevance.
   
   - `knowledge_graph` (bool): If set to `True`, the system utilizes a knowledge graph to enhance understanding and connections between concepts.
   
   - `kv_pairs` (bool): When enabled, the system extracts and utilizes key-value pairs from the input, which can be useful for structured data processing.
   
   - `llm_model_name` (str): Specifies the name of the language model to be used, in this case "solar-pro".

These options allow for fine-tuning of the RAG system's behavior, potentially improving the relevance and accuracy of responses based on the specific use case and requirements.

In [17]:
# Import necessary libraries
from langchain_upstage import ChatUpstage as Chat

# Initialize the ChatUpstage model
# Use the previously defined rag_model_name and rag_base_url
chat = Chat(
    api_key=os.environ["UPSTAGE_API_KEY"],
    model=rag_model_name,
    base_url=rag_base_url,
    extra_body={
        "hybrid_search": True,
        "contextual_query": True,
        "contextual_chunk": True,
        "knowledge_graph": True,
        "kv_pairs": True,
        "llm_model_name": "solar-pro",
    },
)

# Send a query to the chat model
question = "What's the Document AI?"
response = chat.invoke(question)

# Print the response content in a formatted way
rprint(response)

AIMessage(
    content='Document AI is a product provided by Upstage. It consists of three solutions: Layout Analyzer, Key 
Information Extractor, and Document OCR.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': 'sung-pro-1001',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-82511478-2c70-4f45-b892-7a70d3b77551-0'
)

In [18]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[{"role": "user", "content": "What's Solar 10.7B LLM MMLU?"}],  # The user's query
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_chunk": False,
        "contextual_query": True,
        "knowledge_graph": False,
        "kv_pairs": False,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='The text states that "SOLAR 10.7 11B Pretrained 66.04 61.95 84.60 65.48 45.04 83.66 55.50"
in the MetaMathQA task, which is part of the MMLU framework. The MMLU score for SOLAR 10.7B is "83.66".',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727794827.0,
    model='sung-pro-1001',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 13,
                'filename': 'solar_paper',
                'original_chunk': 'research and practical applications, fostering\nwider accessibility and utility 
in various fields.• Superior Performance Across Diverse\nBenchmarks: SOLAR 10.7B excels in var-\nious benchmarks, 
outperforming established\nmodels like Llama 2 and Mistral 7B in reason-\ning, mathematics, and the MMLU 
framework.• Advancement in Instruction-Following Ca-\npabilities: The introduction of SOLAR 10.7B-\nInstruct, a 
variant fine-tuned for enhanced\ninstruction-following abilities, marks a sig-\nnificant improvement in the model’s
ability to\nunderstand and execute complex instructions.Sanghoon Kim, Dahyun Kim, Chanjun Park,\nWonsung Lee, Wonho
Song, Yunsu Kim and\nHyeonwoo Kim contributed equally to this paper.\nSanghoon Kim led the Foundation Model 
part,\nwith Dahyun Kim, Wonho Song, Yunsu Kim, and\nHyeonwoo Kim. Chanjun Park led the Data and\nEvaluation 
(Data-Centric LLM) part, with Yungi\nKim, Jihoo Kim, Changbae Ahn, Seonghoon Yang,\nSukyung Lee, and Hyunbyung 
Park. Wonsung Lee',
                'is_contextual_chunk': False,
                '_id': '47e1c16a-7ba9-4d1d-a8cc-87dc914685b7',
                '_collection_name': 
'18d8ae4e61837fd00bd88a1be2ecc540e137c59819407f8d3f1c94458943ff7f_sung-pro-1001',
                'contextual_query': '"What is Solar 10.7B LLM MMLU?"'
            },
            page_content='research and practical applications, fostering\nwider accessibility and utility in 
various fields.• Superior Performance Across Diverse\nBenchmarks: SOLAR 10.7B excels in var-\nious benchmarks, 
outperforming established\nmodels like Llama 2 and Mistral 7B in reason-\ning, mathematics, and the MMLU 
framework.• Advancement in Instruction-Following Ca-\npabilities: The introduction of SOLAR 10.7B-\nInstruct, a 
variant fine-tuned for enhanced\ninstruction-following abilities, marks a sig-\nnificant improvement in the model’s
ability to\nunderstand and execute complex instructions.Sanghoon Kim, Dahyun Kim, Chanjun Park,\nWonsung Lee, Wonho
Song, Yunsu Kim and\nHyeonwoo Kim contributed equally to this paper.\nSanghoon Kim led the Foundation Model 
part,\nwith Dahyun Kim, Wonho Song, Yunsu Kim, and\nHyeonwoo Kim. Chanjun Park led the Data and\nEvaluation 
(Data-Centric LLM) part, with Yungi\nKim, Jihoo Kim, Changbae Ahn, Seonghoon Yang,\nSukyung Lee, and Hyunbyung 
Park. Wonsung Lee',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 13,
                'filename': 'solar_paper',
                'original_chunk': 'Wen, and Jiawei Han. 2023. Don’t make your llm\nan evaluation benchmark cheater.
arXiv preprint\narXiv:2311.01964.Daniel M Ziegler, Nisan Stiennon, Jeffrey Wu, Tom B\nBrown, Alec Radford, Dario 
Amodei, Paul Chris-\ntiano, and Geoffrey Irving. 2019. Fine-tuning lan

In [19]:
# Contextual Query

from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)
# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[
        {"role": "user", "content": "What's the benefits of Document AI?"},
        {"role": "assistant", "content": "It can help you understand the Document AI"},
        {"role": "user", "content": "How about its features?"},  # The user's query
    ],
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "contextual_query": True,
        "contextual_chunk": False,
        "knowledge_graph": False,
        "kv_pairs": False,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='Document AI is comprised of three solutions:\n1. "Layout Analyzer: Extracts layouts, 
tables, and figures from any document."\n2. "Key Information Extractor: Extracts key information from target 
documents."\n3. "Document OCR: Extracts all text from any document."',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727794844.0,
    model='sung-pro-1001',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 1,
                'filename': 'docai',
                'original_chunk': 'Document AI: Digitize anythingUpstage Document AI is comprised of three 
solutions:● Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor:
Extracts key information from target documents.\n● Document OCR: Extracts all text from any document.In various 
enterprise use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous
manual human work for document\nprocessing. Due to its strong performance, the solution has been heavily adopted by
various\nleading insurance providers, yielding significant productivity gains and operating cost\nreductions.Please
find customer testimonials, key customers, and customer success stories below.[Key customers]',
                'is_contextual_chunk': False,
                '_id': '6132beaa-f3cb-478c-804d-78114cd8347c',
                '_collection_name': 
'18d8ae4e61837fd00bd88a1be2ecc540e137c59819407f8d3f1c94458943ff7f_sung-pro-1001',
                'contextual_query': '"What about Document AI\'s features?"'
            },
            page_content='Document AI: Digitize anythingUpstage Document AI is comprised of three solutions:● 
Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor: Extracts 
key information from target documents.\n● Document OCR: Extracts all text from any document.In various enterprise 
use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous manual 
human work for document\nprocessing. Due to its strong performance, the solution has been heavily adopted by 
various\nleading insurance providers, yielding significant productivity gains and operating cost\nreductions.Please
find customer testimonials, key customers, and customer success stories below.[Key customers]',
            type='Document'
        ),
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 16,
                'filename': 'kim-tse-2008',
                'original_chunk': '[42] M.D. Penta, S. Gradara, and G. Antoniol, “Traceability Recovery\nin RAD 
Software Systems,” Proc. 10th IEEE Int’l Workshop Program\nComprehension, pp. 207-216, 2002.\n[43] B. Raskutti, 
H.L. Ferra´ , and A. Kowalczyk, “Second-Order\nFeatures for Maximizing Text Classification Performance,” 
Proc.\n12th European Conf. Machine Learning, pp. 419-430, 2001.\n[44] Scientific Toolworks, “Maintenance, 
Understanding, Metrics and\nDocumentation Tools for Ada, C, C++, Java, and FORTRAN,”\nhttp://www.scitools.com/, 
2005.\n[45] S. Scott and S. Matwin, “Feature Engineering for Text Classifica-\ntion,” Proc. 16th Int’l Conf. 
Machine Learning, pp. 379-388, 1999.[46] F. Sebastiani, “Machine Le

In [20]:
from openai import OpenAI

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=rag_base_url,  # Using the custom RAG API base URL
    api_key=os.environ[
        "UPSTAGE_API_KEY"
    ],  # Using the API key from environment variables
)

# Send a request to the RAG model
response = client.chat.completions.create(
    model=rag_model_name,  # Using the previously defined RAG model name
    messages=[{"role": "user", "content": "Explain Document AI"}],  # The user's query
    stream=False,  # Disable streaming for synchronous response
    extra_body={
        "hybrid_search": True,  # Enable hybrid search for better results
        "contextual_query": True,  # Enable contextual querying
        "knowledge_graph": True,
        "kv_pairs": True,
        "verbose": True,  # Request verbose output for debugging
    },
)

# Print the model's response
rprint(response)

ChatCompletion(
    id='_',
    choices=[
        Choice(
            finish_reason=None,
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='"Upstage Document AI is a comprehensive document processing solution, as indicated by its 
name. It consists of three solutions: Layout Analyzer, Key Information Extractor, and Document OCR. These solutions
have been successfully applied in various enterprise use cases, particularly in the insurance sector, leading to 
higher accuracy rates (95%-99%) compared to manual work for document processing. The company has experienced 
significant productivity gains, operational cost reductions, and widespread adoption among leading insurance 
providers."',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1727794857.0,
    model='sung-pro-1001',
    object='chat.completion',
    service_tier=None,
    system_fingerprint=None,
    usage=[
        CompletionUsage(
            completion_tokens=None,
            prompt_tokens=None,
            total_tokens=None,
            completion_tokens_details=None,
            id=None,
            metadata={
                'total_pages': 1,
                'filename': 'docai',
                'contextual_chunk': 'Context: The main topic of the document is "Upstage Document AI," which is 
described as comprising three solutions: Layout Analyzer, Key Information Extractor, and Document OCR. These 
solutions have been successfully applied in various enterprise use cases, particularly in the insurance sector, 
resulting in higher accuracy rates (95%-98%) compared to manual human work for document processing. This strong 
performance has led to significant productivity gains, operating cost reductions, and widespread adoption among 
leading insurance providers.',
                'original_chunk': 'Document AI: Digitize anythingUpstage Document AI is comprised of three 
solutions:● Layout Analyzer: Extracts layouts, tables, and figures from any document.\n● Key Information Extractor:
Extracts key information from target documents.\n● Document OCR: Extracts all text from any document.In various 
enterprise use cases, Upstage Document AI resulted in higher accuracy\n(95%-98% accuracy rate) compared to previous
manual human work for document\nprocessing. Due to its strong performance, the solution has been heavily adopted by
various\nleading insurance providers, yielding significant productivity gains and operating cost\nreductions.Please
find customer testimonials, key customers, and customer success stories below.[Key customers]',
                'is_contextual_chunk': True,
                'kv_pairs': [
                    {'key': 'Document Topic', 'value': 'Upstage Document AI'},
                    {
                        'key': 'Document AI Components',
                        'value': 'Layout Analyzer, Key Information Extractor, Document OCR'
                    },
                    {
                        'key': 'Layout Analyzer Function',
                        'value': 'Extracts layouts, tables, and figures from any document'
                    },
                    {
                        'key': 'Key Information Extractor Function',
                        'value': 'Extracts key information from target documents'
                    },
                    {'key': 'Document OCR Function', 'value': 'Extracts all text from any document'},
                    {'key': 'Accuracy Rate Compared to Manual Work', 'value': '95%-98%'},
                    {'key': 'Primary Sector of Use', 'value': 'Insurance'},
                    {
                        'key': 'Benefits of Upstage Document AI',
                        'value': 'Productivity gains, operating cost reductions'
                    }
                ],
                'knowledge_graph': {
                  

In [21]:
# Initialize the chat model using the public model name (accessible to everyone)
chat = Chat(model=public_model_name, base_url=rag_base_url)

# Define the question about Document AI and its usage
question = "What's Document AI? How can it be used?"

# Invoke the chat model with the question
response = chat.invoke(question)
rprint(response)

AIMessage(
    content='The information is not available in the given sources.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-826fbbcc-e2c4-4804-8f2f-9516287195e7-0'
)

In [22]:
# RAG API with history
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)
chat = Chat(model=public_model_name, base_url=rag_base_url)

chain = prompt_template | chat

questions = [
    "What's the benefits of Document AI?",
    "How about its features?",
    "Why we need it?",
]

chat_history = []

for question in questions:
    print("Question: ", question)

    human_message = HumanMessage(content=question)
    response = chain.invoke({"input": question, "chat_history": chat_history})
    rprint(response)
    chat_history.append(human_message)
    chat_history.append(response)

# Print the chat history
print("Chat History:")
for chat in chat_history:
    rprint(chat)

Question:  What's the benefits of Document AI?


AIMessage(
    content='According to the key-value pairs provided, the benefits of Upstage Document AI are "Productivity 
gains, operating cost reductions".',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-00408a06-09e0-49f9-932c-a3705f09d68d-0'
)

Question:  How about its features?


AIMessage(
    content='The features of Document AI are:\n\n1. Extracts layouts, tables, and figures from any document.\n2. 
Extracts key information from target documents.\n3. Extracts all text from any document.\n\nThese features are 
attributed to the components of Document AI: Layout Analyzer, Key Information Extractor, and Document OCR.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-ab422469-da04-4d01-82dc-3153586b7e71-0'
)

Question:  Why we need it?


...

AIMessage(
    content='According to the key-value pairs provided, the benefits of Upstage Document AI are "Productivity 
gains, operating cost reductions".',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-00408a06-09e0-49f9-932c-a3705f09d68d-0'
)

HumanMessage(content='How about its features?', additional_kwargs={}, response_metadata={})

AIMessage(
    content='The features of Document AI are:\n\n1. Extracts layouts, tables, and figures from any document.\n2. 
Extracts key information from target documents.\n3. Extracts all text from any document.\n\nThese features are 
attributed to the components of Document AI: Layout Analyzer, Key Information Extractor, and Document OCR.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-ab422469-da04-4d01-82dc-3153586b7e71-0'
)

HumanMessage(content='Why we need it?', additional_kwargs={}, response_metadata={})

AIMessage(
    content='The information is not available in the given sources.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': None,
        'model_name': '36127e427ea30842d7f1',
        'system_fingerprint': None,
        'finish_reason': None,
        'logprobs': None
    },
    id='run-550db5ab-139b-47d4-9c3f-843364b005c0-0'
)

In [23]:
# Public mode name is read-only
rprint(f"Try to upload a file to the public model...{public_model_name}")

# Expect 400 BadRequest Error
# If you want to upload a file to the public model, you need to use the private model name

try:
    file_response = client.files.create(
        file=open("pdfs/solar_paper.pdf", "rb"),
        purpose="assistants",
        extra_body={"model_name": public_model_name},
    )
except Exception as e:
    rprint(e)

Try to upload a file to the public model...36127e427ea30842d7f1

Error code: 400 - {'detail': "Invalid model name. Please use only alphanumeric characters, hyphens, and 
underscores. Example: 'my-model-123'"}